In [1]:
import pandas as pd
import numpy as np

[Pandas cookbook](https://pandas.pydata.org/docs/user_guide/cookbook.html): a repository for short and sweet examples and links for useful pandas recipes.

# 1. Data Structures

## 1.1 Series 

pandas.Series is one-dimensional **ndarray** with **axis labels**. So it acts like an **ndarray** and a **dict**.

### 1.1.1 Create Series 

In [2]:
# Construct function
pd.Series(data=None, index=None, dtype=None, name=None, copy=False)
# Parameter data: array-like, Iterable, dict, scalar
# Parameter index: array-like or Index(1d)

# create Series from ndarray
s = pd.Series(np.random.randn(5))
print(s)

# create Series from array
s = pd.Series([2, 3, 1, 4, 5], index = ['a', 'b', 'c', 'd', 'e'])
print(s)

# create Series from dict
s = pd.Series({'a': 1.0, 'b': 2.0, 'c': 3.0}) # key as the index
print(s)
s = pd.Series({'a': 1.0, 'b': 2.0, 'c': 3.0}, index=['x', 'y', 'a', 'a']) # Non-unique index values are allowed.
# If an index is passed, the values in data corresponding to the labels in the index will be pulled out.
# Return NaN if no key corresponds to the index label.
print(s)

# create Series from scalar
s = pd.Series(5, index=['a', 'b', 'c'])
print(s)


0    1.988128
1    0.073344
2   -1.240947
3   -0.883062
4   -0.188685
dtype: float64
a    2
b    3
c    1
d    4
e    5
dtype: int64
a    1.0
b    2.0
c    3.0
dtype: float64
x    NaN
y    NaN
a    1.0
a    1.0
dtype: float64
a    5
b    5
c    5
dtype: int64


### 1.1.2 Series is ndarray-like 

Series acts very similarly to a ndarray and is a valid argument to most Numpy functions. And operations such as slicing will also slice the index.

In [3]:
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'], name='mySeries')
print(s[:3])
print(s['b':'d']) # different from number slicing, index slicing includes both start point and end point.
print(s[s > s.median()])

a   -0.716633
b    1.793208
c    0.945213
Name: mySeries, dtype: float64
b    1.793208
c    0.945213
d   -0.904667
Name: mySeries, dtype: float64
b    1.793208
c    0.945213
Name: mySeries, dtype: float64


Different from ndarray, operations between Series automatically align the data based on label. Thus, you can write computations without giving consideration to whether the Series involved have the same labels. If a label is not found in one Series or the other, the result will be marked as missing NaN.

In [4]:
s[1:] + s[:-1]

a         NaN
b    3.586415
c    1.890425
d   -1.809334
e         NaN
Name: mySeries, dtype: float64

While Series is ndarray-like, if you need an actual ndarray, use Series.to_numpy()

In [5]:
s.to_numpy()

array([-0.71663258,  1.79320767,  0.94521269, -0.9046672 , -0.58324401])

### 1.1.3 Series is dict-like 

In [6]:
print('a' in s)
print(s['a'])
print(s.get('a'))
print('f' in s)
# print(s['f']) # error
print(s.get('f'))
print(s.get('f', np.nan))

True
-0.7166325776763249
-0.7166325776763249
False
None
nan


## 1.2 DataFrame

pandas.DataFrame is a two-dimensional, size-multable, potentially heterogeneous tabular data. It contains labeled axes (rows and columns). Can be thought of as a dict-like container for Series objects. The primary pandas data structure.

### 1.2.1 Create DataFrame 

In [7]:
# Construct function
pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
# Parameter data: ndarry(structured or homogeneous), Iterable, dict, or DataFrame

# create DataFrame from dict of Series
df = pd.DataFrame({'col1': pd.Series([1.0, 2.0, 3.0], index=['a', 'b', 'c']),
                  'col2': pd.Series([4.0, 5.0, 6.0], index=['a', 'b', 'd'])})
print(df)

# create DataFrame from dict of ndarrays/lists
df = pd.DataFrame({'col1':[1, 2, 3], 'col2':[4, 5, 6]}, index=['a', 'b', 'c'])
print(df)

# Optional
# create DataFrame from a list of dicts.
df = pd.DataFrame([{'a': 1, 'b': 2}, {'a': 4, 'b': 5, 'c': 6}])
print(df)

df = pd.DataFrame([{'a': 1, 'b': 2}, {'a': 4, 'b': 5, 'c': 6}], columns=['col1', 'col2'])
print(df)

# To summarize, keys in the dict data are for columns

   col1  col2
a   1.0   4.0
b   2.0   5.0
c   3.0   NaN
d   NaN   6.0
   col1  col2
a     1     4
b     2     5
c     3     6
   a  b    c
0  1  2  NaN
1  4  5  6.0
   col1  col2
0   NaN   NaN
1   NaN   NaN


### 1.2.2 Read and write DataFrame

 ### 1. ***pd.read_csv()***, read csv file & text file, return DataFrame

**Parameters:**  
**filepath_or_buffer**: filepath, str, url, or any object with a read() method (such as an open file or StringIO)

**sep / delimiter**: sep, default ','; delimiter, default None

**header**: default 'infer'. header=0: infer from first line. header=None: column names are passed. header='infer': if column names are passed, like header=None, else like header = 0. Note that this parameter ignores commented lines and empty lines if skip_blank_lines=True(default), so header=0 denotes the first line of data rather than the first line of the file.
**names**: array-like, default None. Column names.
**index_col**: int, str, sequence of int/str. Column(s) to use as row labels.

**usecols**: list-like or callable, default None. Return a subset of columns.
**skip_rows**: list-like or integer, default None. Line numbers to skip(0-indexed) or number of lines to skip (int) at the start of the file.
**nrows**: int, defatul None. Number of rows of file to read.

**true_values**: list, default None. Values to consider as True.
**false_values**: list, default None. Values to consider as False.
**na_values**: list and others, defaule None. Additional strings to recognize as NA/NaN.
**keep_default_na**: boolean, defatult True. Whether or not to include the default NaN values when parsing the data.

**parse_dates**: boolean or list of ints or names or list of lists or dict, default False.  
- If True: try parsing the index.  
- If [1, 2, 3]:try parsing columns 1,2,3 each as a separate data column. 
- If [[1, 3]]: combine columns 1 and 3 and parse as a single data column.
**date_format**: str or dict of column -> format, default None. If used in conjuction with parse_dates, will parse dates according to this format. For anything more complex, please read in as object and then apply to_datetime() as-needed.

**dtype**: indicate the date type for the whole DataFrame or individual columns.
**converters**: deal with columns with mixed types.

In [8]:
from io import BytesIO
data = b"word,length\n" b"Tr\xc3\xa4umen,7\n" b"Gr\xc3\xbc\xc3\x9fe,5"
df = pd.read_csv(BytesIO(data), encoding='utf-8') # By test, adding encoding='utf-8' or not won't affect the result
df

,word,length
0,Träumen,7
1,Grüße,5


### 2. to_csv(), Series/DataFrame instance method, store objects to csv/text file
**Parameters:**  
**path_or_buf**: a string path to the file or a file object.  
**sep**: Field delimiter, default ','.

In [9]:
from io import StringIO
with StringIO() as f:
    df.to_csv(f)
    f.seek(0)
    print(f.read())

,word,length
0,Träumen,7
1,Grüße,5



### 1.2.3 Explore DataFrame data

In [10]:
s = pd.Series(np.random.randn(5), index=["a", "b", "c", "d", "e"])
df = pd.DataFrame(np.random.randn(8, 3), index=list('abcdefgh'), columns=["A", "B", "C"])

In [11]:
df.head() # display 5 head rows
df.head(10) # display 10 head rows
df.tail() # display 5 tail rows
df.tail(10) # display 10 tail rows

,A,B,C
a,-0.595104,-0.735907,2.045119
b,0.269366,1.692751,1.544135
c,1.956259,-1.152488,-1.047629
d,0.264113,-1.467085,-0.320092
e,-1.379760,-0.335331,-0.970484
f,-0.708503,0.993003,0.663502
g,0.064109,-0.740489,-0.740096
h,-1.480373,-0.840697,0.237957


In [12]:
df.shape

(8, 3)

In [13]:
df.index
df.columns

Index(['A', 'B', 'C'], dtype='object')

In [14]:
s.describe()
df.describe()

,A,B,C
count,8.000000,8.000000,8.000000
mean,-0.201237,-0.323280,0.176551
std,1.108459,1.095774,1.165206
min,-1.480373,-1.467085,-1.047629
25%,-0.876317,-0.918645,-0.797693
50%,-0.265497,-0.738198,-0.041068
75%,0.265426,-0.003248,0.883661
max,1.956259,1.692751,2.045119


# 2. Indexing and selecting data 

In [15]:
df

,A,B,C
a,-0.595104,-0.735907,2.045119
b,0.269366,1.692751,1.544135
c,1.956259,-1.152488,-1.047629
d,0.264113,-1.467085,-0.320092
e,-1.379760,-0.335331,-0.970484
f,-0.708503,0.993003,0.663502
g,0.064109,-0.740489,-0.740096
h,-1.480373,-0.840697,0.237957


## 2.1 []

In [16]:
df['A']

a   -0.595104
b    0.269366
c    1.956259
d    0.264113
e   -1.379760
f   -0.708503
g    0.064109
h   -1.480373
Name: A, dtype: float64

In [17]:
# df['a'] # KeyError

In [18]:
df.A

a   -0.595104
b    0.269366
c    1.956259
d    0.264113
e   -1.379760
f   -0.708503
g    0.064109
h   -1.480373
Name: A, dtype: float64

In [19]:
df['A']['a']

-0.5951035075457793

In [20]:
df['A'][0]

-0.5951035075457793

## 2.2 .loc 

**Label based**, but may laso be used with a boolean array. ***.loc*** will raise KeyError when the items are not found. Allowed inputs are:  
- A single label, e.g. 5 or 'a' (Note that 5 is interpreted as a label of the index. This use is not an integer position along the index).  
- A list or array of labels ['a', 'b', 'c'].  
- A slice object with labels ['a':'f'] (Note that contrary to usual Python slices, both the start and the stop are included, when present in the index.)   
- A boolean array (any NA values will be treated as False).  
- A callable function with one argument (the calling Series or DataFrame) and that returns valid output for indexing (one of the above).  
- For multi-axes selection, axes left out of the specification are assumed to be **:**, e.g. p.loc['a'] is equivalent to p.loc['a', :].

In [21]:
# df.loc[3] # KeyError
df.loc['a']

A   -0.595104
B   -0.735907
C    2.045119
Name: a, dtype: float64

In [22]:
# df.loc['A'] # KeyError
df.loc[:, 'A'] # same with df['A']

a   -0.595104
b    0.269366
c    1.956259
d    0.264113
e   -1.379760
f   -0.708503
g    0.064109
h   -1.480373
Name: A, dtype: float64

In [23]:
df.loc['a', 'B':'C']

B   -0.735907
C    2.045119
Name: a, dtype: float64

In [24]:
df.loc['a':'b'] # same with df.loc['a':'b', :]

,A,B,C
a,-0.595104,-0.735907,2.045119
b,0.269366,1.692751,1.544135


In [25]:
df.loc['a':'z']

,A,B,C
a,-0.595104,-0.735907,2.045119
b,0.269366,1.692751,1.544135
c,1.956259,-1.152488,-1.047629
d,0.264113,-1.467085,-0.320092
e,-1.379760,-0.335331,-0.970484
f,-0.708503,0.993003,0.663502
g,0.064109,-0.740489,-0.740096
h,-1.480373,-0.840697,0.237957


In [26]:
# Selection by boolean array
print(df.loc[:, 'A'] < 0)
df.loc[df.loc[:, 'A'] < 0] # same with df.log[df.loc[:, 'A'] < 0, :]

a     True
b    False
c    False
d    False
e     True
f     True
g    False
h     True
Name: A, dtype: bool


,A,B,C
a,-0.595104,-0.735907,2.045119
e,-1.379760,-0.335331,-0.970484
f,-0.708503,0.993003,0.663502
h,-1.480373,-0.840697,0.237957


In [27]:
mask = pd.array([True, False, True, False, True, False, True, pd.NA])
print(mask)
df.loc[pd.array(mask)]

<BooleanArray>
[True, False, True, False, True, False, True, <NA>]
Length: 8, dtype: boolean


,A,B,C
a,-0.595104,-0.735907,2.045119
c,1.956259,-1.152488,-1.047629
e,-1.379760,-0.335331,-0.970484
g,0.064109,-0.740489,-0.740096


In [28]:
# Selection by callable
df.loc[lambda df: df['A'] < 0] # callable returns a boolean array

,A,B,C
a,-0.595104,-0.735907,2.045119
e,-1.379760,-0.335331,-0.970484
f,-0.708503,0.993003,0.663502
h,-1.480373,-0.840697,0.237957


In [29]:
df.loc[:, lambda df: ['A', 'C']] # callable returns a list of labels

,A,C
a,-0.595104,2.045119
b,0.269366,1.544135
c,1.956259,-1.047629
d,0.264113,-0.320092
e,-1.379760,-0.970484
f,-0.708503,0.663502
g,0.064109,-0.740096
h,-1.480373,0.237957


In [30]:
df.loc[lambda df:list('abcde'), :] # callable returns a list of labels

,A,B,C
a,-0.595104,-0.735907,2.045119
b,0.269366,1.692751,1.544135
c,1.956259,-1.152488,-1.047629
d,0.264113,-1.467085,-0.320092
e,-1.379760,-0.335331,-0.970484


In [31]:
# return an element
df.loc['a']['C']

2.0451188326994862

## 2.3 .iloc 

**Integer position based** (from 0 to length-1 of the axis), but may also be used with a boolean array. ***.iloc*** will raise IndexError if a requested indexer is out-of-bounds, except slice indexers which allow out-of-bounds indexing (this conforms with Python/NumPy slice semantics). Allowed inputs are (similar to ***.loc***):  
- An integer e.g. 5.
- A list or array of integers [4, 3, 0]
- A slice object with ints 1:7.
- A boolean array (any NA values will be treated as False).
- A callable function with one argument (the calling Series or DataFrame) and that returns valid output for indexing (one of the above).
- For multi-axes selection, axes left out of the specification are assumed to be :, e.g. p.loc['a'] is equivalent to p.loc['a', :].

In [32]:
df

,A,B,C
a,-0.595104,-0.735907,2.045119
b,0.269366,1.692751,1.544135
c,1.956259,-1.152488,-1.047629
d,0.264113,-1.467085,-0.320092
e,-1.379760,-0.335331,-0.970484
f,-0.708503,0.993003,0.663502
g,0.064109,-0.740489,-0.740096
h,-1.480373,-0.840697,0.237957


In [33]:
df.iloc[2]

A    1.956259
B   -1.152488
C   -1.047629
Name: c, dtype: float64

In [34]:
df.iloc[[5, 3, 1]]

,A,B,C
f,-0.708503,0.993003,0.663502
d,0.264113,-1.467085,-0.320092
b,0.269366,1.692751,1.544135


In [35]:
df.iloc[3:5, 1:]

,B,C
d,-1.467085,-0.320092
e,-0.335331,-0.970484


In [36]:
df.iloc[:, [True, False, True]]

,A,C
a,-0.595104,2.045119
b,0.269366,1.544135
c,1.956259,-1.047629
d,0.264113,-0.320092
e,-1.379760,-0.970484
f,-0.708503,0.663502
g,0.064109,-0.740096
h,-1.480373,0.237957


In [37]:
print(df.B > 0)
# df.iloc[df.B > 0] #ValueError: iLocation based boolean indexing cannot use an indexable as a mask
mask = pd.array([True, False, True, False, True, False, pd.NA, pd.NA])
print(mask)
df.iloc[mask]

a    False
b     True
c    False
d    False
e    False
f     True
g    False
h    False
Name: B, dtype: bool
<BooleanArray>
[True, False, True, False, True, False, <NA>, <NA>]
Length: 8, dtype: boolean


,A,B,C
a,-0.595104,-0.735907,2.045119
c,1.956259,-1.152488,-1.047629
e,-1.379760,-0.335331,-0.970484


In [38]:
# Selection by callable
df.iloc[lambda df: [4, 5, 6]]

,A,B,C
e,-1.379760,-0.335331,-0.970484
f,-0.708503,0.993003,0.663502
g,0.064109,-0.740489,-0.740096


In [39]:
df.iloc[lambda df: [4, 5, 6], lambda df: [2, 1]]

,C,B
e,-0.970484,-0.335331
f,0.663502,0.993003
g,-0.740096,-0.740489


## 2.4 Selecting random samples 

Use ***sample()*** method to select random rows or columns from a Series or DataFrame. 
  
***DataFrame.sample(n=1, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False)***    
**Parameters:**
- **n**: int, optional. Number of items from axis to return. Default = 1.
- **frac**: float, optional. Fraction of axis items to return.
- **replace**: bool, default False. With or without replacement. Allow or disallow sampling of the same row more than once.
- **weights**: str or ndarray-like, optional. Default 'None' results in equal probability weighting. If called on DataFrame and axis=0, can accept the name of a column as the weights array. Otherwise, accept ndarray-like. If passed a Series as weights, will align with target object on index. Index values in weights not found in sampled object will be ignored and index values in sampled object not in weights will be assigned weights of zero. Unless weights are a Series, weights must be same length as axis being sampled. If weights do not sum to 1, they will be normalized to sum to 1. 
- **random_state**: int (as a seed) or a Numpy RandomState object. Random number generator.
- **axis**: 0 or 'index', 1 or 'columns'. Axis to sample. Default is index for DataFrame.

In [40]:
df

,A,B,C
a,-0.595104,-0.735907,2.045119
b,0.269366,1.692751,1.544135
c,1.956259,-1.152488,-1.047629
d,0.264113,-1.467085,-0.320092
e,-1.379760,-0.335331,-0.970484
f,-0.708503,0.993003,0.663502
g,0.064109,-0.740489,-0.740096
h,-1.480373,-0.840697,0.237957


In [41]:
df.sample()

,A,B,C
f,-0.708503,0.993003,0.663502


In [42]:
df.sample(3, random_state=4)

,A,B,C
e,-1.379760,-0.335331,-0.970484
h,-1.480373,-0.840697,0.237957
d,0.264113,-1.467085,-0.320092


In [43]:
df.sample(3, random_state=4)

,A,B,C
e,-1.379760,-0.335331,-0.970484
h,-1.480373,-0.840697,0.237957
d,0.264113,-1.467085,-0.320092


In [44]:
df.sample(frac=0.5)

,A,B,C
c,1.956259,-1.152488,-1.047629
h,-1.480373,-0.840697,0.237957
d,0.264113,-1.467085,-0.320092
e,-1.379760,-0.335331,-0.970484


In [45]:
df.sample(n=3, axis=1, replace=True, weights=[1,0,0])

,A,A,A
a,-0.595104,-0.595104,-0.595104
b,0.269366,0.269366,0.269366
c,1.956259,1.956259,1.956259
d,0.264113,0.264113,0.264113
e,-1.379760,-1.379760,-1.379760
f,-0.708503,-0.708503,-0.708503
g,0.064109,0.064109,0.064109
h,-1.480373,-1.480373,-1.480373


# 3. Merge, join, concatenate

We use ***pandas.concat*** to concatenate pandas objects along a paticular axis and allow optional set logic along the other axes. Can also add a layer of hierarchical indexing on the concatenation axis, which may be useful if the labels are the same (or overlapping) on the passed axis number.  
***pandas.concat(objs, *, axis=0, join='outer', ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=False, copy=None)***  
**Parameters:**
- **objs**: a sequence or mapping of Series or DataFrame objects.
- **axis**: {0/'index', 1/'columns'}, default 0
- **join**: {'inner', 'outer'}, default 'outer'
- **ignore_index**: bool, default False. If True, do not use the index values along the concatenation axis. The resulting axis will be labeled 0,...,n-1.
- **keys**: sequence, default None. Construct hierarchical index using the passed keys as the outermost level.  
  
**Returns:**  
When concatenating all Series along the index (axis=0), a Series is returned. When objs contains at least one DataFrame, a DataFrame is returned. When concatenating along the columns (axis=1), a DataFrame is returned.

In [46]:
df1 = pd.DataFrame({
    'A': ['A0', 'A1', 'A2', 'A3'],
    'B': ['B0', 'B1', 'B2', 'B3'],
    'C': ['C0', 'C1', 'C2', 'C3']}, index=[0, 1, 2, 3])
df2 = pd.DataFrame({
    'A': ['A4', 'A5', 'A6'],
    'B': ['B4', 'B5', 'B6'],
    'C': ['C4', 'C5', 'C6']}, index=[4, 5, 6])
df3 = pd.DataFrame({
    'B': ['B2', 'B3', 'B6'],
    'D': ['D2', 'D3', 'D6'],
    'F': ['F2', 'F3', 'F6']}, index=[2, 3, 6])

In [47]:
df1

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2
3,A3,B3,C3


In [48]:
df2

,A,B,C
4,A4,B4,C4
5,A5,B5,C5
6,A6,B6,C6


In [49]:
pd.concat([df1, df2])

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2
3,A3,B3,C3
4,A4,B4,C4
5,A5,B5,C5
6,A6,B6,C6


In [50]:
pd.concat([df1, df2], axis=1)

,A,B,C,A,B,C
0,A0,B0,C0,NaN,NaN,NaN
1,A1,B1,C1,NaN,NaN,NaN
2,A2,B2,C2,NaN,NaN,NaN
3,A3,B3,C3,NaN,NaN,NaN
4,NaN,NaN,NaN,A4,B4,C4
5,NaN,NaN,NaN,A5,B5,C5
6,NaN,NaN,NaN,A6,B6,C6


In [51]:
df3

,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6


In [52]:
pd.concat([df1, df3], join='inner', axis=1)

,A,B,C,B,D,F
2,A2,B2,C2,B2,D2,F2
3,A3,B3,C3,B3,D3,F3


In [53]:
result = pd.concat([df1, df3], axis=1, join='inner', keys=['x', 'y'])
result

x           y        
    A   B   C   B   D   F
2  A2  B2  C2  B2  D2  F2
3  A3  B3  C3  B3  D3  F3

In [54]:
result = pd.concat({'x': df1, 'y': df3}, axis=1, join='inner') # the same with above cell
result

x           y        
    A   B   C   B   D   F
2  A2  B2  C2  B2  D2  F2
3  A3  B3  C3  B3  D3  F3

In [55]:
result.x.loc[2, 'B']

'B2'

In [56]:
df1

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2
3,A3,B3,C3


In [57]:
# Append rows to a DataFrame
s2 = pd.Series(['A4', 'B4', 'C4'], index=['A', 'B', 'C'])
pd.concat([df1, s2.to_frame().T], ignore_index=True)

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2
3,A3,B3,C3
4,A4,B4,C4


# 4. Group by: split-apply-combine

By “group by” we are referring to a process involving one or more of the following steps:
- **Splitting** the data into groups based on some criteria.
- **Applying** a function to each group independently.
- **Combining** the results into a data structure.

## 4.1 Splitting data into groups

### 4.1.1 groupby function
Group DataFrame using a mapper or by a Series of columns.  
***DataFrame.groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, observed=False, dropna=True)***  
**Parameters:** 
- **by**: **mapping, function, label, pd.Grouper or list of such**. If by is a function, it's called on each value of the object's index. If a dict or Series is passed, the Series or dict VALUES will be used to determine the groups. If a list or ndarray of length equal to the selected axis is passed, the values are used as-is to determine the groups. A label or list of labels may be passed to group by the columns in self.
- **axis**: **{0/'index', 1/'columns'}, default 0**. Split along rows (0) or columns (1).
- **level**: **int, level name, or sequence of such, default None**. If the axis is a MultiIndex (hierarchical), group by a particular level or levels. Do not specify both by and level.
- **sort**: **bool, default True**. Sort group keys. Get better performance by turining this off. Note this does not influence the order of observations within each group. Groupby preserves the order of rows within each group.
- **dropna**: **bool, default True**. If True, and if group keys contain NA values, NA values together with row/column will be dropped. If False, NA values will also be treated as the key in groups.  
  
Series also has groupby function.

In [59]:
df = pd.DataFrame(
    {
        "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
        "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
        "C": np.random.randn(8),
        "D": np.random.randn(8),
    })
df

,A,B,C,D
0,foo,one,-1.170722,-0.276401
1,bar,one,0.026203,-0.991155
2,foo,two,-1.002403,-1.501484
3,bar,three,-0.232708,-0.063161
4,foo,two,-0.169243,-0.940424
5,bar,two,-0.674804,0.876703
6,foo,one,-1.337081,-0.732849
7,foo,three,-0.056083,0.708644


In [99]:
# Iterate and print through groups
def print_groups(grouped):
    for name, group in grouped:
        print(name)
        print(group)

In [129]:
# Group by a column label
print_groups(df.groupby('A'))

bar
     A      B         C         D
1  bar    one  0.026203 -0.991155
3  bar  three -0.232708 -0.063161
5  bar    two -0.674804  0.876703
foo
     A      B         C         D
0  foo    one -1.170722 -0.276401
2  foo    two -1.002403 -1.501484
4  foo    two -0.169243 -0.940424
6  foo    one -1.337081 -0.732849
7  foo  three -0.056083  0.708644


In [113]:
# Group by a list of labels
print_groups(df.groupby(['A', 'B']))

('bar', 'one')
     A    B         C         D
1  bar  one  0.026203 -0.991155
('bar', 'three')
     A      B         C         D
3  bar  three -0.232708 -0.063161
('bar', 'two')
     A    B         C         D
5  bar  two -0.674804  0.876703
('foo', 'one')
     A    B         C         D
0  foo  one -1.170722 -0.276401
6  foo  one -1.337081 -0.732849
('foo', 'three')
     A      B         C         D
7  foo  three -0.056083  0.708644
('foo', 'two')
     A    B         C         D
2  foo  two -1.002403 -1.501484
4  foo  two -0.169243 -0.940424


In [149]:
df.groupby(['A', 'B']).mean()

C         D
A   B                        
bar one    0.026203 -0.991155
    three -0.232708 -0.063161
    two   -0.674804  0.876703
foo one   -1.253902 -0.504625
    three -0.056083  0.708644
    two   -0.585823 -1.220954

In [101]:
# Group by function
print_groups(df.groupby(lambda x: x % 2)) # the function is called on each value of the index of selected axis.

0
     A    B         C         D
0  foo  one -1.170722 -0.276401
2  foo  two -1.002403 -1.501484
4  foo  two -0.169243 -0.940424
6  foo  one -1.337081 -0.732849
1
     A      B         C         D
1  bar    one  0.026203 -0.991155
3  bar  three -0.232708 -0.063161
5  bar    two -0.674804  0.876703
7  foo  three -0.056083  0.708644


In [102]:
# Group columns by function
def get_letter_type(letter):
    if letter.lower() in 'aeiou':
        return 'vowel'
    else:
        return 'consonant'
print_groups(df.groupby(get_letter_type, axis=1))

consonant
       B         C         D
0    one -1.170722 -0.276401
1    one  0.026203 -0.991155
2    two -1.002403 -1.501484
3  three -0.232708 -0.063161
4    two -0.169243 -0.940424
5    two -0.674804  0.876703
6    one -1.337081 -0.732849
7  three -0.056083  0.708644
vowel
     A
0  foo
1  bar
2  foo
3  bar
4  foo
5  bar
6  foo
7  foo


In [107]:
# Group by list or ndarray of the same length
print_groups(df.groupby([1, 0, 0, 1], axis=1))

0
       B         C
0    one -1.170722
1    one  0.026203
2    two -1.002403
3  three -0.232708
4    two -0.169243
5    two -0.674804
6    one -1.337081
7  three -0.056083
1
     A         D
0  foo -0.276401
1  bar -0.991155
2  foo -1.501484
3  bar -0.063161
4  foo -0.940424
5  bar  0.876703
6  foo -0.732849
7  foo  0.708644


In [127]:
# Optional
# Group by series
print_groups(df.groupby(pd.Series([1,2,1,3,1,1,2,3], index=np.arange(8))))
# Group by dict
print_groups(df.groupby({'A':'aa', 'B':'bb', 'C':'bb', 'D':'aa'}, axis=1))

1
     A    B         C         D
0  foo  one -1.170722 -0.276401
2  foo  two -1.002403 -1.501484
4  foo  two -0.169243 -0.940424
5  bar  two -0.674804  0.876703
2
     A    B         C         D
1  bar  one  0.026203 -0.991155
6  foo  one -1.337081 -0.732849
3
     A      B         C         D
3  bar  three -0.232708 -0.063161
7  foo  three -0.056083  0.708644
aa
     A         D
0  foo -0.276401
1  bar -0.991155
2  foo -1.501484
3  bar -0.063161
4  foo -0.940424
5  bar  0.876703
6  foo -0.732849
7  foo  0.708644
bb
       B         C
0    one -1.170722
1    one  0.026203
2    two -1.002403
3  three -0.232708
4    two -0.169243
5    two -0.674804
6    one -1.337081
7  three -0.056083


In [133]:
# GroupBy dropna
df_list = [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]
df_dropna = pd.DataFrame(df_list, columns=['a', 'b', 'c'])
df_dropna

,a,b,c
0,1,2.0,3
1,1,NaN,4
2,2,1.0,3
3,1,2.0,2


In [134]:
print_groups(df_dropna.groupby('b', dropna=True))

1.0
   a    b  c
2  2  1.0  3
2.0
   a    b  c
0  1  2.0  3
3  1  2.0  2


### 4.1.2 Selecting a group

In [131]:
grouped = df.groupby('A')
grouped.groups

{'bar': [1, 3, 5], 'foo': [0, 2, 4, 6, 7]}

In [132]:
grouped.get_group('bar')

,A,B,C,D
1,bar,one,0.026203,-0.991155
3,bar,three,-0.232708,-0.063161
5,bar,two,-0.674804,0.876703


## 4.2 Aggregation 

An aggregation is a GroupBy operation that reduces the dimension of the grouping object. The result of an aggregation is, or at least is treated as, a scalar value for each column in a group.

In [136]:
animals = pd.DataFrame({'kind': ['cat', 'dog', 'cat', 'dog'], 'height': [9.1, 6.0, 9.5, 34.0], 'weight': [7.9, 7.5, 9.9, 198.0]})
animals

,kind,height,weight
0,cat,9.1,7.9
1,dog,6.0,7.5
2,cat,9.5,9.9
3,dog,34.0,198.0


In [158]:
grouped = animals.groupby('kind')
gmean = grouped.mean()

,height,weight
kind,,
cat,9.3,8.90
dog,20.0,102.75


In [145]:
grouped['height'].idxmax()

kind
cat    2
dog    3
Name: height, dtype: int64

In [146]:
animals.iloc[grouped['height'].idxmax().array]

,kind,height,weight
2,cat,9.5,9.9
3,dog,34.0,198.0


In [147]:
animals.iloc[grouped['height'].idxmax().to_numpy()]

,kind,height,weight
2,cat,9.5,9.9
3,dog,34.0,198.0


In [148]:
grouped.describe()

height                                               weight           
      count  mean        std  min   25%   50%   75%   max  count    mean   
kind                                                                       
cat     2.0   9.3   0.282843  9.1   9.2   9.3   9.4   9.5    2.0    8.90  \
dog     2.0  20.0  19.798990  6.0  13.0  20.0  27.0  34.0    2.0  102.75   

                                                       
             std  min     25%     50%      75%    max  
kind                                                   
cat     1.414214  7.9   8.400    8.90    9.400    9.9  
dog   134.703842  7.5  55.125  102.75  150.375  198.0

In [150]:
# Aggregation with user-defined functions
animals

,kind,height,weight
0,cat,9.1,7.9
1,dog,6.0,7.5
2,cat,9.5,9.9
3,dog,34.0,198.0


In [152]:
animals.groupby('kind')['height'].agg(lambda x:set(x))

kind
cat     {9.1, 9.5}
dog    {34.0, 6.0}
Name: height, dtype: object

In [153]:
animals.groupby('kind')[['weight', 'height']].agg(lambda x: x.astype(int).sum())

,weight,height
kind,,
cat,16,18
dog,205,40


In [154]:
# Applying multiple functions at once
animals.groupby('kind').agg(['sum', 'mean', 'std'])

height                  weight                    
        sum  mean        std    sum    mean         std
kind                                                   
cat    18.6   9.3   0.282843   17.8    8.90    1.414214
dog    40.0  20.0  19.798990  205.5  102.75  134.703842

## 4.3 Flexible ***apply***

In [193]:
animals.groupby('kind').apply(lambda x: x > x.mean())

height  weight
kind                  
cat  0   False   False
     2    True    True
dog  1   False   False
     3    True    True

In [204]:
def select_higher_than_average(group): # the user defined function's 
    return group[group['height'] > group['height'].mean()]

animals.groupby('kind').apply(select_higher_than_average)      

,,kind,height,weight
kind,,,,
cat,2,cat,9.5,9.9
dog,3,dog,34.0,198.0
